In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carregando os dados

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
WORK_DIR = '/content/drive/My Drive/datasets'
DATA_DIR = os.path.join(WORK_DIR, 'olist')

In [4]:
df_abt = pd.read_csv(os.path.join(DATA_DIR, 'propensao_revenda_abt.csv'))

In [5]:
df_train = df_abt.query('data_ref_safra < "2018-03-01"')

df_oot = df_abt.query('data_ref_safra == "2018-03-01"')

key_vars = ['data_ref_safra', 'seller_id']
num_vars = ['tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia']
cat_vars = ['uf']
target = 'nao_revendeu_next_6m'
features = cat_vars + num_vars

# Dados de treinamento
X = df_train[features]
y = df_train[target]

# Dados de avaliação (out of time)
X_oot = df_oot[features]
y_oot = df_oot[target]

In [6]:
print('Conjunto de Dados:', X.shape)
print('Conjunto de Teste:', X_oot.shape)

Conjunto de Dados: (3495, 6)
Conjunto de Teste: (1874, 6)


## Pipeline utilizado

Vamos utilizar o estimador LogisticRegression para testar todos os cenários

In [7]:
!pip install feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 3.9 MB/s eta 0:00:00


In [8]:
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper

# Criando Pipelines
rl = Pipeline(steps=[
    ('numeric_imputer', MeanMedianImputer(variables=num_vars)),
    ('std', SklearnTransformerWrapper(transformer=StandardScaler(), variables=num_vars)),
    ('categoric_imputer', CategoricalImputer(variables=cat_vars)),
    ('one_hot_encoder', OneHotEncoder(variables=cat_vars)),
    ('logistic_regression', LogisticRegression(random_state=42))
])

## Funções para avaliar as técnicas

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score
def avaliar_tecnica(cv_results):
  mean_accuracy = np.array(cv_results).mean()
  std_accuracy  = np.array(cv_results).std()
  print(f"Acurácia Média com CV: {mean_accuracy:.3f}")
  print(f"Desvio-Padrão com CV: {std_accuracy:.3f}")

# K-Fold Cross-Validation

Para essa técnica não é necessário criar o conjunto de validação, uma vez que a própria técnica irá criar as partes (folds), portanto iremos utilizar o conjunto de dados `X` e `y` que contém os dados de treino

In [10]:
X.head(3)

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia
0,SP,3,3,1,2685.00,74
1,ES,171,207,9,21275.23,2
2,SP,38,42,15,781.80,2


In [11]:
# Importando o objeto que separa os nossos dados de acordo com o KFold
from sklearn.model_selection import KFold

# Criando o objeto KFold com k=5, aleatorizando as linhas antes do split com shuffle=True
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
X.head()

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia
0,SP,3,3,1,2685.00,74
1,ES,171,207,9,21275.23,2
2,SP,38,42,15,781.80,2
3,GO,1,1,1,120.00,16
4,SP,130,141,75,16228.88,8


In [13]:
# Cria uma nova coluna nos dados de treino que irá armazenar o valor
# da nova coluna que guardará o valor a qual fold pertence cada linha
X["kfold"] = -1

# kf.split() retorna os índices de cada fold.
# Criando o conjunto de treino e validação.
# Fazemos uma iteração sobre cada fold e atribuímos a cada linha qual fold ela pertence
for fold, (trn_, val_) in enumerate(kf.split(X=X)):
    X.loc[val_, 'kfold'] = fold

<ipython-input-13-542280f1006d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["kfold"] = -1


In [14]:
X.head(5)

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,kfold
0,SP,3,3,1,2685.00,74,0
1,ES,171,207,9,21275.23,2,4
2,SP,38,42,15,781.80,2,2
3,GO,1,1,1,120.00,16,3
4,SP,130,141,75,16228.88,8,4


In [15]:
# distribuição da variável target em cada fold.
# o KFold não mantém a distribuição original da variável target.
pd.crosstab(X['kfold'], y, normalize='index')

nao_revendeu_next_6m,0,1
kfold,,
0,0.632332,0.367668
1,0.589413,0.410587
2,0.628040,0.371960
3,0.629471,0.370529
4,0.615165,0.384835


In [16]:
# distribuição da variável target
df_abt['nao_revendeu_next_6m'].value_counts(1)

0    0.61762
1    0.38238
Name: nao_revendeu_next_6m, dtype: float64

**Calculando a acurácia com CV**

In [17]:
from sklearn.model_selection import cross_val_score

In [18]:
# definindo uma estratégia de kfold com k=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# usando a função cross_val_score() para treinar e validar o modelo em cada fold
# aqui usamos a acurácia como métrica a ser calculada em cada hold-out set dentro de cada fold
cv_results = cross_val_score(estimator=rl, X=X[features], y=y, scoring='accuracy', cv=kf, n_jobs=-1)

In [19]:
# mostrando os resultados: perceba que temos um array com 5 valores, já que temos 5 folds (k=5)
cv_results

array([0.83834049, 0.7925608 , 0.84549356, 0.82260372, 0.82832618])

In [20]:
# Calculando a acurácia média e o desvio-padrão médio
avaliar_tecnica(cv_results)

Acurácia Média com CV: 0.825
Desvio-Padrão com CV: 0.018


# Stratified K-Fold Cross-Validation

Aqui iremos mostrar como aplicar cross-validation usando a estratégia de Stratified Cross-Validation.

In [21]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [22]:
X["skfold"] = -1

for fold, (trn_, val_) in enumerate(skf.split(X=X, y=y)):
    X.loc[val_, 'skfold'] = fold

<ipython-input-22-f7ae0a73fc7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["skfold"] = -1


In [23]:
pd.crosstab(X['skfold'], y, normalize='index')

nao_revendeu_next_6m,0,1
skfold,,
0,0.618026,0.381974
1,0.618026,0.381974
2,0.619456,0.380544
3,0.619456,0.380544
4,0.619456,0.380544


Agora, diferentemente da KFold, temos a mesma distribuição da variável target em todos os folds.

**Calculando a acurácia com CV**

In [24]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_val_score(estimator=rl, X=X[features], y=y, scoring='accuracy', cv=skf, n_jobs=-1)

In [25]:
cv_results

array([0.82689557, 0.81688126, 0.81688126, 0.81545064, 0.84978541])

In [26]:
# calculando a acurácia média e o desvio-padrão médio
avaliar_tecnica(cv_results)

Acurácia Média com CV: 0.825
Desvio-Padrão com CV: 0.013
